## Setup

In [1]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.34 🚀 Python-3.10.0 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)
Setup complete ✅ (16 CPUs, 11.4 GB RAM, 438.7/476.1 GB disk)


In [2]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.19.0


In [3]:
MODEL = "yolov8x.pt"
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [1]

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [4]:
import os
import csv
from tqdm.notebook import tqdm
import numpy as np
import math
import torch
from IPython import display
import subprocess


## Utils

In [5]:
import VARIABLES2
import importlib
importlib.reload(VARIABLES2)
from VARIABLES2 import *

In [8]:
import VARIABLES2
import importlib
importlib.reload(VARIABLES2)
from VARIABLES2 import *

from utils import *

videos_folder = "probando"
videos_folder_path = os.path.join("full_recordings", videos_folder)
data = [["file_name", "in", "out"]]

prev_in, prev_out = 0, 0

vih = VideoInfoHandler()


with open(os.path.join("results", f"{videos_folder}.csv"), "w", newline="") as csv_output:
  writer = csv.writer(csv_output)
  try:
    for file_name in tqdm(os.listdir(videos_folder_path), desc=" Videos remaining", position=0):
      file_path = os.path.join(videos_folder_path, file_name)
      vih.re_init(file_path)
      process_video(
        source_path = file_path,
        target_path = f"results/videos/probando/{file_name}",
        model=model,
        selected_classes=selected_classes,
        callback=callback,
        vih=vih
      )
      max_in = max(vih.line_zones, key=lambda x: x.in_count)
      max_out = max(vih.line_zones, key=lambda x: x.out_count)
      
      data.append([file_name, max_in.in_count - prev_in, max_out.out_count - prev_out])
      prev_in = max_in.in_count
      prev_out = max_out.out_count
      
  except KeyError as e:
    print(e)
  finally: 
    writer.writerows(data)
    result = subprocess.run(f"echo {max_in.in_count}, {max_out.out_count} > results/{videos_folder}.txt", shell=True, capture_output=True, text=True)

    # Check if the command was successful
    if result.returncode == 0:
        print(result.stdout)
    else:
        print("Command failed with error:")
        print(result.stdout)
        print(result.stderr)
    print(max_in.in_count, max_out.out_count)



 Videos remaining:   0%|          | 0/2 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/365 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/365 [00:00<?, ?it/s]


0 4
